# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman (*RSA*) cryptosystem demonstrated below combines secure key&ndash;exchange with digital signature to provide confidentiality, integrity and authentication in a message&ndash;exchange between two parties (Alice and Bob) over an insecure channel. A third party (Eve) eavesdropping on this channel has no practical way of learning the content of the message, nor any way of corrupting it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality, and utilizes a *key&ndash;encapsulation mechanism* (or *KEM*). This differs from the more traditional approach to key&ndash;exchange, based on the *discrete logarithm problem*, introduced by Diffie and Hellman in their 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf) (see the notebooks [*DH.ipynb*](https://nbviewer.org/github/dchampion/crypto/blob/master/doc/DH.ipynb) and [*EC.ipynb*](https://nbviewer.org/github/dchampion/crypto/blob/master/doc/EC.ipynb) for demonstrations of classic Diffie&ndash;Hellman and *elliptic&ndash;curve* Diffie&ndash;Hellman, respectively).

Note that the *KEM* presented here lacks a property the Diffie&ndash;Hellman approaches do not, and which some might consider a disqualifying deficiency; this property is known as [*perfect forward secrecy*](https://en.wikipedia.org/wiki/Forward_secrecy). To overcome this deficiency, the digital signature component of the module demonstrated below may nevertheless be used in concert with Diffie&ndash;Hellman key exchange to implement a robust cryptosystem (the *KEM* implementation is used here only to demonstrate RSA encryption).

The digital signature component of this cryptosystem provides for message integrity and authentication, concepts that were also introduced by Diffie and Hellman in their 1976 paper. However, it was not until Rivest, Shamir and Adleman published their seminal 1978 paper [*A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems*](https://people.csail.mit.edu/rivest/Rsapaper.pdf) that a practical implementation of it was presented.

## The Source Code
The following sequence of function calls to the `rsa` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice signs and encrypts a private message, and sends the message to Bob over an insecure channel. Meanwhile, an adversary&mdash;Eve&mdash;intercepts all the information exchanged between Alice and Bob on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `rsa` module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must *not* be transmitted over the insecure channel).

Explanations of behavior appear *below* the code snippets and their outputs.

In [1]:
import sys
sys.path.append("../code/src")
import rsa
sys.path.append("../code/test")
import sym

Import the `rsa` module, and a module `sym` that implements an elementary symmetric cipher that Alice and Bob will use in the protocol.

In [2]:
pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

The sequence begins with Alice generating her RSA key parameters. She does this by calling the `rsa.generate_rsa_key` function, passing it a modulus size of `2048` bits. The function returns the quintuple *pA*, *qA*, *[nA]*, *d3A* and *d5A* (the suffix *A* on all the parameters identify them as belonging to Alice). Alice transmits her public modulus *[nA]* to Bob.

Alice must keep the values *pA* and *qA*&mdash;the 1024&ndash;bit prime factors of *[nA]*&mdash;secret. She must also keep *d3A* and *d5A*&mdash;her signing and decryption keys, respectively&mdash;secret.

To get an idea of the size and proportions of these parameters, let's print some of them out.

In [3]:
print(pA)

173299652527378621849678293235644969530349924813106301112035832066464015229640674143152804958294250780106645367583514294433931316536472757674174491910666392917034505278393027701595287970946808478199125579027898496130469536479327229549981254585425386154736674542208776136649634810224950027072534168858654726967


In [4]:
print(qA)

136865116631199199914911844211218584622312717029340328685300645662427237001918849312161419955976665842479718932776734164534931792525015636195650497182257393266003086746295646067564669293433517636261173153094517947191215764516850627196819826577390529318516397566044372106921874590223725180385370507505992847179


In [5]:
print(nA)

23718677155305970276189970542606743912653400430584102326069153337388370839571800962005761865798095003038703177058041240373118234612785557499282408538067729542901109881844874843529874345142979957954700188037686726158035929748015269831327583006530108155267805192455593496712343839554066246545880135775961217367014634390871218048424605949890404899111573544791657698245312442853802703196535253854832624360051454272779018872584987916019037211685627787254838712325899821952708524512006716251450785969325365716639982927239867295319375942485155515587009827778529700457871708365369920439236015967851006980772424382739201176093


 Note that *[nA]*, which is the product of *pA* and *qA*, is twice the length of each individually; *pA*'s and *qA*'s ~310 decimal digits to *[nA]*'s ~620 decimal digits.

Recall that Alice requested a modulus of 2048 bits in length in her call to `rsa.generate_rsa_key`. Next, let's print the bit lengths of *[nA]* and its prime factors.

In [6]:
print(nA.bit_length())

2048


In [7]:
print(pA.bit_length())

1024


In [8]:
print(qA.bit_length())

1024


What we see is that a decimal number of ~620 digits equates to a bit&ndash;length of 2048, and further that a decimal number of ~310 digits is 1024 bits long.

Next, let's check to see if *[nA]* is indeed the product of *pA* and *qA* (note this step is not necessary for the protocol; it is done here just for demonstration purposes).

In [9]:
print(qA * pA == nA)

True


Now it's Bob's turn to generate RSA keys of his own. To differentiate Bob's keys from Alice's keys, we append Bob's with the captial letter *B*.

In [10]:
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Because Bob requested a modulus size of 2048, based on our inspection of Alice's keys above we can conclude that Bob's keys are the same size as Alice's keys; so we don't need to print their values here.

Now that Alice and Bob have generated their keys, and transmitted to each other their public moduli *[nA]* and *[nB]*, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob.

In [11]:
mA = "Sign and encrypt me!"

First, Alice stores the message "Sign and encrypt me!" in the variable `mA`.

In [12]:
oA = rsa.sign(d3A, pA, qA, mA)

Then Alice calls the `rsa.sign` fuction, passing it her secret signing key *d3A*, the two factors of *[nA]*, *pA* and *qA*, and the message *mA*, to sign the message.

She stores the signature in the variable *[oA]*`.

In [13]:
KA, cA = rsa.encrypt_random_key(nB, rsa.ENCRYPTION_EXPONENT)

Next, Alice calls the `rsa.encrypt_random_key` function, passing it Bob's RSA modulus *[nB]* and the global encryption exponent *[rsa.ENCRYPTION_EXPONENT]*. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., 3DES, AES). The function returns the pair *KA* and *[cA]*, which are, respectively, the actual encryption key, and the ciphertext of its input material; the material Bob will use to replicate the same encryption key on his side of the channel.

Alice must keep the session key *KA* private, but she transmits the ciphertext *[cA]* to Bob.

In [14]:
mAC = sym.encrypt(KA, mA)

Then Alice encrypts the private message calling the symmetric cipher function `sym.encrypt`, passing it her symmetric key *KA* and the plaintext of her message *mA* (note that in a real application, Alice and Bob would use an industrial&ndash;strength cipher (e.g., AES, 3DES) for message encryption).

The result of this operation is the ciphertext *[mAC]*.

Alice transmits the message signature *[oA]*, the message ciphertext *[mAC]* and the ciphertext of the symmetric&ndash;key's input material *[cA]* to Bob.

In [15]:
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

In order to decrypt the message, Bob must reproduce the symmetric key Alice used to encrypt it. He does this by calling  `rsa.decrypt_random_key`, passing it his private decryption key *d5B*, the ciphertext of Alice's symmetric&ndash;key input material *[cA]*, and the two prime factors of his public modulus *pB* and *qB*.

In [16]:
print(KB == KA)

True


Here we make sure the key Bob decrypted above, *KB*, is the same as the key Alice encrypted, *KA* (again, this step is not required for the protocol; it is only used here to demonstrate that the keys are indeed equal).

In [17]:
mB = sym.decrypt(KB, mAC)

Bob calls `sym.decrypt`, passing it *KB* and the ciphertext of Alice's message *[mAC]*, to recover Alice's plaintext message *mA*.

Bob stores the result in *mB*.

In [18]:
print(mB == mA)

True


Here we prove that the recovered plaintext indeed matches Alice's original message.

In [19]:
print(mB)

Sign and encrypt me!


Let's print the message, just to make sure.

All that remains to complete the session is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

In [20]:
print(rsa.verify(nA, rsa.VERIFICATION_EXPONENT, mB, oA))

True


Bob calls the `rsa.verify` fuction, passing Alice's public modulus *[nA]*, the constant signature&ndash;verification key *[rsa.VERIFICATION_EXPONENT]*, the decrypted message *mB* and the sigature sent to him by Alice *[oA]*.

If the assertion above holds, we've proved that Bob has successfully decrypted and verified the message sent to him by Alice.